# **Creating Model**

In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
data = pd.read_csv('/content/drive/MyDrive/Dataset/IMD.csv')

In [3]:
data.head()

review sentiment
0  I grew up (b. 1965) watching and loving the Th...  negative
1  When I put this movie in my DVD player, and sa...  negative
2  Why do people who do not know what a particula...  negative
3  Even though I have great interest in Biblical ...  negative
4  Im a die hard Dads Army fan and nothing will e...  positive

In [4]:
data['sentiment'] = data['sentiment'].map({'positive': 1, 'negative': 0})

data['review'] = data['review'].str.replace('<[^<]+?>', '')
data['review'] = data['review'].str.replace('http\S+', '')
data['review'] = data['review'].str.replace('[^A-Za-z0-9 ]+', '')
data['review'] = data['review'].str.lower()

<ipython-input-4-988738ba6cb3>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  data['review'] = data['review'].str.replace('<[^<]+?>', '')
<ipython-input-4-988738ba6cb3>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  data['review'] = data['review'].str.replace('http\S+', '')
<ipython-input-4-988738ba6cb3>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  data['review'] = data['review'].str.replace('[^A-Za-z0-9 ]+', '')


In [5]:
nan_values = data['sentiment'].isna().sum()
print(nan_values)

0


In [6]:
X_train, X_test, y_train, y_test = train_test_split(data['review'], data['sentiment'], test_size=0.2, random_state=42)

In [7]:
max_features = 10000
maxlen = 200
embedding_dim = 100
batch_size = 32
epochs = 10

In [8]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_train)

In [9]:
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)

In [10]:
model = Sequential()
model.add(Embedding(max_features, embedding_dim, input_length=maxlen))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(64))
model.add(Dense(1, activation='sigmoid'))

In [11]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 100)          1000000   
                                                                 
 lstm (LSTM)                 (None, 200, 64)           42240     
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 1075329 (4.10 MB)
Trainable params: 1075329 (4.10 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Epoch 1/10
800/800 [==============================] - 231s 284ms/step - loss: 0.4177 - accuracy: 0.8087 - val_loss: 0.3646 - val_accuracy: 0.8475
Epoch 2/10
800/800 [==============================] - 226s 283ms/step - loss: 0.2624 - accuracy: 0.8950 - val_loss: 0.3332 - val_accuracy: 0.8691
Epoch 3/10
800/800 [==============================] - 233s 291ms/step - loss: 0.1858 - accuracy: 0.9305 - val_loss: 0.3381 - val_accuracy: 0.8583
Epoch 4/10
800/800 [==============================] - 222s 278ms/step - loss: 0.1454 - accuracy: 0.9463 - val_loss: 0.3805 - val_accuracy: 0.8677
Epoch 5/10
800/800 [==============================] - 219s 274ms/step - loss: 0.1211 - accuracy: 0.9556 - val_loss: 0.3815 - val_accuracy: 0.8580
Epoch 6/10
800/800 [==============================] - 232s 290ms/step - loss: 0.0942 - accuracy: 0.9680 - val_loss: 0.4209 - val_accuracy: 0.8547
Epoch 7/10
800/800 [==============================] - 230s 288ms/step - loss: 0.0562 - accuracy: 0.9823 - val_loss: 0.5926 -

In [13]:
y_pred = model.predict(X_test)
y_pred = np.round(y_pred).flatten()

250/250 [==============================] - 29s 111ms/step


In [14]:
acc = accuracy_score(y_test, y_pred)

In [15]:
print(f'Accuracy on the test set: {acc:.3f}')

Accuracy on the test set: 0.844


# **Validating the Model**

In [16]:
import re
import numpy as np
from keras.preprocessing.sequence import pad_sequences

In [17]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub('[^A-Za-z0-9 ]+', '', text)
    return text

In [18]:
def tokenize_and_pad(text, tokenizer, maxlen):
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=maxlen)
    return padded_sequence

In [19]:
def predict_sentiment(model, text_sequence):
    prediction = model.predict(text_sequence)
    binary_prediction = np.round(prediction).flatten()
    return binary_prediction

In [20]:
new_reviews = [
    "I absolutely loved this movie! The acting was superb.",
    "The product was terrible, and I regret buying it.",
    "Amazing experience, I highly recommend it!",
    "Disappointing service, wouldn't go back.",
]

In [21]:
new_reviews_preprocessed = [preprocess_text(review) for review in new_reviews]
new_reviews_sequences = [tokenize_and_pad(review, tokenizer, maxlen) for review in new_reviews_preprocessed]

In [22]:
for i, review_sequence in enumerate(new_reviews_sequences):
    prediction = predict_sentiment(model, review_sequence)
    sentiment = "Positive" if prediction == 1 else "Negative"
    print(f"Review {i + 1}: {sentiment} sentiment")

1/1 [==============================] - 2s 2s/step
Review 1: Positive sentiment
1/1 [==============================] - 0s 135ms/step
Review 2: Positive sentiment
1/1 [==============================] - 0s 126ms/step
Review 3: Positive sentiment
1/1 [==============================] - 0s 95ms/step
Review 4: Negative sentiment


# **Validating by Taking User Input**

In [23]:
user_input = input("Enter a review: ")

Enter a review: Amazing Series is this. Love it


In [24]:
user_input_preprocessed = preprocess_text(user_input)
user_input_sequence = tokenize_and_pad(user_input_preprocessed, tokenizer, maxlen)

In [25]:
prediction = predict_sentiment(model, user_input_sequence)
sentiment = "Positive" if prediction == 1 else "Negative"
print(f"The sentiment of the review is: {sentiment}")

1/1 [==============================] - 0s 178ms/step
The sentiment of the review is: Positive
